In [ ]:
from textvec import vectorizers
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize 
import nltk
from collections import defaultdict
import pandas as pd
import nltk
import csv
import glob, pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report   
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import GridSearchCV
%matplotlib inline


vocab = 8000


data = pd.read_csv('/home/oem/jin/python/after_prepro10yr.csv', encoding="CP949")




X = data.iloc[:,1].values
y = data.iloc[:,2].values


tficf_vec = vectorizers.TfIcfVectorizer(sublinear_tf=True)



count_vec = CountVectorizer(max_features=vocab,
    token_pattern=r'\w{1,}').fit(X)
tfnt_vect = TfidfVectorizer(max_features=vocab).fit(X)

tficf_vec.fit(count_vec.transform(X), y)
icf = tficf_vec.transform(count_vec.transform(X))

to_icf_vect = icf.toarray()
tol_tf_vect = tfnt_vect.transform(X).toarray()
tol_cout_vect = count_vec.transform(X).toarray()

print(tol_tf_vect[1])

ifidficf = []


for line in range(0, len(to_icf_vect)):
    lin_va = []
    #print('totla',line)
    for wo_val in range(0, vocab):
        
        tficf = tol_tf_vect[line][wo_val]*to_icf_vect[line][wo_val]/tol_cout_vect[line][wo_val] if tol_cout_vect[line][wo_val]!= 0 else 0
        lin_va.append(tficf)
    ifidficf.append(lin_va)
    

        

X_totl = pad_sequences(ifidficf,dtype='float32',padding='post')
    

X_train, X_test, y_train, y_test = train_test_split(X_totl, y, test_size=0.25, shuffle=True, random_state=25, stratify=y)



In [ ]:
##### deault ML capplication #######


svc = SVC()
LogReg = LogisticRegression()
nb = MultinomialNB()
rf = RandomForestClassifier()
xgb = XGBClassifier()
model = [svc, LogReg, nb, rf, xgb]


for m in model:
    m.fit(X_train, y_train)
    

for m in model:
    pred = m.predict(X_test)
    print('Acc for model', accuracy_score(y_test,pred))
    print('Confusion Matrix for model', confusion_matrix(y_test, pred))
    print('F1-score for model', classification_report(y_test, pred))
    
##### end of deault ML capplication #######    

In [ ]:
##### gridsearchCV for SVM classifer 


svm_tf_idf_icf = open('tf_idf_icf_svm.txt', 'w')

parameters = [{"kernel":["rbf"], "gamma":[0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05,0.1, 1, 5, 10, 20, 50, 100, 200, 500, 1000], "C":[0.001, 0.005, 0.01, 0.1, 1, 5, 10, 20, 30, 40, 50, 100, 200, 500, 700, 1000], "degree":[1, 2, 3, 4, 5]}]

#scores = ["precision", "recall_macro"]
scores = ["accuracy"]

print("# tuning hypter parmeter for %s" % score)
print()
    
clf_svm = GridSearchCV(SVC(random_state = 2), parameters, scoring=scores, verbose = 3, n_jobs=6, cv=5)
clf_svm.fit(X_train, y_train)
    
print("best parameters st found on devleopment set:")
print()
print(clf_svm.best_params_)
print()
print("Grid scores on development set:")
means = clf_svm.cv_results_["mean_test_score"]
stds = clf_svm.cv_results_["std_test_score"]
    

for mean, std, params in tqdm(zip(means, stds, clf_svm.cv_results_["params"])):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=svm_tf_idf_icf)

print()

print("Detailed classifciaiton reprot:")
print()
print("the model is trained on the full development set.")
print("the scores are computed on the full developement set")
print()
y_true, y_pred = y_test, clf_svm.predict(X_test)
print(classification_report(y_true, y_pred), file=svm_tf_idf_icf)

print()

svm_tf_idf_icf.close()





###### end gridsearchCV for SVM classifer #######

In [ ]:
##### gridsearchCV for naive_bayes classifer #######

rb_idf_icf_tf = open('tf_idf_icf_rb.txt', 'w')


parameters = [{"alpha": [00.00001, 0.0001, 0.001, 0.01, 0.1, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 8, 9, 10, 20, 40, 60, 80, 100, 150, 200, 300, 400, 500], "fit_prior": ["True", "False"]}]
# scores = ["precision", "recall_macro"]
#scores = ["accuracy"]
scores = make_scorer(accuracy_score)

print("# tuning hypter parmeter for %s" % scores)
print()

clf_rb = GridSearchCV(rb, parameters, scoring=scores, verbose=3, n_jobs=6, cv=5)
clf_rb.fit(X_train, y_train)

print("best parameters st found on devleopment set:")
print()
print(clf_rb.best_params_)
print()
print("Grid scores on development set:")
means = clf_rb.cv_results_["mean_test_score"]
stds = clf_rb.cv_results_["std_test_score"]

for mean, std, params in tqdm(zip(means, stds, clf_rb.cv_results_["params"])):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=rb_tf_idf_icf)

print()

print("Detailed classifciaiton reprot:")
print()
print("the model is trained on the full development set.")
print("the scores are computed on the full developement set")
print()
y_true, y_pred = y_test, clf_rb.predict(X_test)
print(classification_report(y_true, y_pred), file=rb_tf_idf_icf)

print()

svm_tf_idf_icf.close()


###### end gridsearchCV for naive_bayes classifer #######

In [ ]:

##### gridsearchCV for ramdon forest classifer #######

rf_idf_icf_tf = open('tf_idf_icf_rf.txt', 'w')

rf = RandomForestClassifier(random_state=2017)
parameters = [{"n_estimators": [4, 6, 8, 10, 15, 20, 40, 60, 80, 100, 150, 200, 300, 400, 500, 600, 800, 1000], "max_features": ['sqrt', 'log2'], "max_depth": [4, 6, 8, 10, 12, 16, 18, 20, 22, 24, 26, 28, 30], 'criterion': ['gini', 'entropy', 'log_loss']}]
scores = make_scorer(accuracy_score)

print("# tuning hypter parmeter for %s" % scores)
print()

clf_rf = GridSearchCV(rf, parameters, scoring=scores, verbose=3, n_jobs=6, cv=5)
clf_rf.fit(X_train, y_train)

print("best parameters st found on devleopment set:")
print()
print(clf_rf.best_params_)
print()
print("Grid scores on development set:")
means = clf_rf.cv_results_["mean_test_score"]
stds = clf_rf.cv_results_["std_test_score"]

for mean, std, params in tqdm(zip(means, stds, clf_rf.cv_results_["params"])):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=rf_tf_idf_icf)

print()

print("Detailed classifciaiton reprot:")
print()
print("the model is trained on the full development set.")
print("the scores are computed on the full developement set")
print()
y_true, y_pred = y_test, clf_rb.predict(X_test)
print(classification_report(y_true, y_pred), file=rf_tf_idf_icf)

print()

rf_tf_idf_icf.close()



##### end gridsearchCV for ramdon forest classifer #######


In [ ]:

##### gridsearchCV for Logistic regression classifer #######


lr_tf_idf_icf = open('tf_idf_icf_lr.txt', 'w')

lr = LogisticRegression(multi_class='multinomial', class_weight='balanced', random_state=2017)

c_range = [0.001, 0.01, 0.1, 1, 10, 100, 200]
max_i = [20, 50, 100, 200, 500, 1000]
parameters = [{"penalty": ["l1"], "C": c_range, "solver": ["saga"], 'max_iter': max_i},
              {"penalty": ["l2"], "C": c_range, "solver": ["saga"], 'max_iter': max_i},
              {"penalty": ["elasticnet"], "C": c_range, "solver": ["saga"], 'max_iter': max_i, 'l1_ratio': [0.5]}]

scores = make_scorer(accuracy_score)

print("# tuning hypter parmeter for %s" % scores)
print()

clf_lr = GridSearchCV(lr, parameters, scoring=scores, verbose=3, n_jobs=6, cv=5)
clf_lr.fit(X_train, y_train)

print("best parameters st found on devleopment set:")
print()
print(clf_lr.best_params_)
print()
print("Grid scores on development set:")
means = clf_lr.cv_results_["mean_test_score"]
stds = clf_lr.cv_results_["std_test_score"]

for mean, std, params in tqdm(zip(means, stds, clf_lr.cv_results_["params"])):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=lr_tf_idf_icf)

print()

print("Detailed classifciaiton reprot:")
print()
print("the model is trained on the full development set.")
print("the scores are computed on the full developement set")
print()
y_true, y_pred = y_test, clf_lr.predict(X_test)
print(classification_report(y_true, y_pred), file=lr_tf_idf_icf)

print()

lr_tf_idf_icf.close()




##### end gridsearchCV for Logistic regression classifer #######



In [ ]:


##### gridsearchCV for XGBoost classifer #######


xgb_tf_idf_icf = open('tf_idf_icf_xgb.txt', 'w')


xgb = XGBClassifier(random_state=2017)
parameters = [{"max_depth":[2, 4, 8, 12, 16, 18, 22, 26], "gamma":[0.1, 1, 2, 4, 8, 12, 16, 20, 24], "colsample_bytree": [0.1, 0.2, 0.4, 0.6],
               "min_child_weight":[1, 2, 4, 8, 10], "subsample":[0.1, 0.5, 0.7, 1]}]
# scores = ["precision", "recall_macro"]
#scores = ["accuracy"]
scores = make_scorer(accuracy_score)

print("# tuning hypter parmeter for %s" % scores)
print()

clf_xgb = GridSearchCV(xgb, parameters, scoring=scores, verbose=3, n_jobs=6, cv=3)
clf_xgb.fit(X_train, y_train)

print("best parameters st found on devleopment set:")
print()
print(clf_xgb.best_params_)
print()
print("Grid scores on development set:")
means = clf_xgb.cv_results_["mean_test_score"]
stds = clf_xgb.cv_results_["std_test_score"]

for mean, std, params in tqdm(zip(means, stds, clf_xgb.cv_results_["params"])):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=xgb_tf_idf_icf)

print()

print("Detailed classifciaiton reprot:")
print()
print("the model is trained on the full development set.")
print("the scores are computed on the full developement set")
print()
y_true, y_pred = y_test, clf_xgb.predict(X_test)
print(classification_report(y_true, y_pred), file=xgb_tf_idf_icf)

print()

xgb_tf_idf_icf.close()


##### end gridsearchCV for XGBoost classifer #######


